In [1]:
import torch
import torch.nn as nn
import time
import numpy as np
import scipy.sparse as sparse
from argparse import ArgumentParser
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.nn.functional as F
import wandb
import pprint
from torch.optim import Adam
from torch.optim import SGD
from torch.optim import Adadelta

use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [2]:
def accuracy(output, labels):
    y_pred = output.max(1)[1].type_as(labels)
    correct = y_pred.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def prepare_dataset(labels, num_classes, configg):
    """ Splits the loaded dataset into train/validation/test sets. """
    if not configg.follow_paper:
        # Follow train/val/test indices as in the official implementation
        # on GitHub: https://github.com/tkipf/pygcn
        train_set = range(140)
        validation_set = range(200, 500)
        test_set = range(500, 1500)
    else:
        # https://arxiv.org/pdf/1609.02907.pdf
        # The original paper proposes that the training set is composed
        # out of 20 samples per class -> 140 samples, but the indices
        # above (range(140)) do not contain 20 samples per class
        # The remaining val/test indices were selected empirically
        classes = [ind for ind in range(num_classes)]
        train_set = []

        # Construct train set (indices) out of 20 samples per each class
        for class_label in classes:
            target_indices = torch.nonzero(labels == class_label, as_tuple=False).tolist()
            train_set += [ind[0] for ind in target_indices[:configg.train_size_per_class]]

        # Extract the remaining samples
        validation_test_set = [ind for ind in range(len(labels)) if ind not in train_set]
        # Split the remaining samples into validation/test set
        validation_set = validation_test_set[:configg.validation_size]
        test_set = validation_test_set[configg.validation_size:configg.validation_size+configg.test_size]

    return train_set, validation_set, test_set


def enumerate_labels(labels):
    """ Converts the labels from the original
        string form to the integer [0:MaxLabels-1]
    """
    unique = list(set(labels))
    labels = np.array([unique.index(label) for label in labels])
    return labels


def normalize_adjacency(adj):
    """ Normalizes the adjacency matrix according to the
        paper by Kipf et al.
        https://arxiv.org/pdf/1609.02907.pdf
    """
    adj = adj + sparse.eye(adj.shape[0])

    node_degrees = np.array(adj.sum(1))
    node_degrees = np.power(node_degrees, -0.5).flatten()
    node_degrees[np.isinf(node_degrees)] = 0.0
    node_degrees[np.isnan(node_degrees)] = 0.0
    degree_matrix = sparse.diags(node_degrees, dtype=np.float32)

    adj = degree_matrix @ adj @ degree_matrix
    return adj


def convert_scipy_to_torch_sparse(matrix):
    matrix_helper_coo = matrix.tocoo().astype('float32')
    data = torch.FloatTensor(matrix_helper_coo.data)
    rows = torch.LongTensor(matrix_helper_coo.row)
    cols = torch.LongTensor(matrix_helper_coo.col)
    indices = torch.vstack([rows, cols])

    shape = torch.Size(matrix_helper_coo.shape)
    matrix = torch.sparse.FloatTensor(indices, data, shape)
    return matrix


def load_data(configg):
    """ Loads the graph data and stores them using
        efficient sparse matrices approach.
    """
    print("Loading Cora dataset...")
    ###############################
    # Loading Graph Nodes Data
    ###############################
    raw_nodes_data = np.genfromtxt(configg.nodes_path, dtype="str")
    raw_node_ids = raw_nodes_data[:, 0].astype('int32')  # unique identifier of each node
    raw_node_labels = raw_nodes_data[:, -1]
    labels_enumerated = enumerate_labels(raw_node_labels)  # target labels as integers
    node_features = sparse.csr_matrix(raw_nodes_data[:, 1:-1], dtype="float32")

    ################################
    # Loading Graph Structure Data
    ################################
    ids_ordered = {raw_id: order for order, raw_id in enumerate(raw_node_ids)}
    raw_edges_data = np.genfromtxt(configg.edges_path, dtype="int32")
    edges_ordered = np.array(list(map(ids_ordered.get, raw_edges_data.flatten())),
                             dtype='int32').reshape(raw_edges_data.shape)
    ####################
    # ADJACENCY MATRIX
    ####################
    adj = sparse.coo_matrix((np.ones(edges_ordered.shape[0]), (edges_ordered[:, 0], edges_ordered[:, 1])),
                            shape=(labels_enumerated.shape[0], labels_enumerated.shape[0]),
                            dtype=np.float32)
    # Make the adjacency matrix symmetric
    adj = adj + adj.T.multiply(adj.T > adj)
    adj = normalize_adjacency(adj)

    ####################################
    # Adapt the data to PyTorch format
    ####################################
    features = torch.FloatTensor(node_features.toarray())
    labels = torch.LongTensor(labels_enumerated)
    adj = convert_scipy_to_torch_sparse(adj)

    print("Dataset loaded.")

    return features, labels, adj, edges_ordered


In [3]:
parser = ArgumentParser()
parser.add_argument("--cuda", type=bool, default=use_cuda)
parser.add_argument("--nodes_path", type=str, default="../data/cora.content")
parser.add_argument("--edges_path", type=str, default="../data/cora.cites")
parser.add_argument("--hidden_dim", type=int, default=16)
parser.add_argument("--dropout", type=float, default=0.5)
parser.add_argument("--use_bias", type=bool, default=True)
parser.add_argument("--train_size_per_class", type=int, default=20)
parser.add_argument("--validation_size", type=int, default=500)
parser.add_argument("--test_size", type=int, default=1000)
parser.add_argument("--lr", type=float, default=1e-2)
parser.add_argument("--weight_decay", type=float, default=5e-3)
parser.add_argument("--patience", type=int, default=10)
parser.add_argument("--epochs", type=int, default=200)
parser.add_argument("--use_early_stopping", type=bool, default=True)
parser.add_argument("--multiple_runs", type=bool, default=False)
parser.add_argument("--num_of_runs", type=int, default=100)
parser.add_argument("--follow_paper", type=bool, default=True)

configg = parser.parse_args("")


In [4]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features, use_bias=True):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(torch.zeros(size=(in_features, out_features))))
        if use_bias:
            self.bias = nn.Parameter(torch.FloatTensor(torch.zeros(size=(out_features,))))
        else:
            self.register_parameter('bias', None)

        self.initialize_weights()

    def initialize_weights(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        x = x @ self.weight
        if self.bias is not None:
            x += self.bias

        return torch.sparse.mm(adj, x)


class GCN(nn.Module):
    def __init__(self, node_features, hidden_dim, num_classes, dropout, use_bias=True):
        super(GCN, self).__init__()
        self.gcn_1 = GCNLayer(node_features, hidden_dim, use_bias)
        self.gcn_2 = GCNLayer(hidden_dim, num_classes, use_bias)
        self.dropout = nn.Dropout(p=dropout)

    def initialize_weights(self):
        self.gcn_1.initialize_weights()
        self.gcn_2.initialize_weights()

    def forward(self, x, adj):
        x = F.relu(self.gcn_1(x, adj))
        x = self.dropout(x)
        x = self.gcn_2(x, adj)
        return x


In [5]:
def training_loop(model, features, labels, adj, train_set_ind, val_set_ind, configg, epochh, optimizerval, lrval):
    if configg.cuda:
        model.cuda()
        adj = adj.cuda()
        features = features.cuda()
        labels = labels.cuda()

    if optimizerval == 'Adam':
        optimizer = Adam(model.parameters(), lr=lrval)
    elif optimizerval == 'SGD':
        optimizer = SGD(model.parameters(), lr=lrval, momentum=0.9)
    elif optimizerval == 'Adadelta':
        optimizer = Adadelta(model.parameters(), lr=lrval, weight_decay=configg.weight_decay)
    criterion = nn.CrossEntropyLoss()

    validation_acc = []
    validation_loss = []

    if configg.use_early_stopping:
        last_min_val_loss = float('inf')
        patience_counter = 0
        stopped_early = False

    t_start = time.time()
    for epoch in range(epochh):
        optimizer.zero_grad()
        model.train()

        y_pred = model(features, adj)
        train_loss = criterion(y_pred[train_set_ind], labels[train_set_ind])
        train_acc = accuracy(y_pred[train_set_ind], labels[train_set_ind])
        train_loss.backward()
        optimizer.step()

        wandb.log({"training_acc": train_acc})
        wandb.log({"training_loss": train_loss.item()})
        
        with torch.no_grad():
            model.eval()
            val_loss = criterion(y_pred[val_set_ind], labels[val_set_ind])
            val_acc = accuracy(y_pred[val_set_ind], labels[val_set_ind])

            validation_loss.append(val_loss.item())
            validation_acc.append(val_acc)
            wandb.log({"epoch": epoch})
            wandb.log({"val_acc": val_acc})
            wandb.log({"val_loss": val_loss.item()})
            if configg.use_early_stopping:
                if val_loss < last_min_val_loss:
                    last_min_val_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter == configg.patience:
                        stopped_early = True
                        t_end = time.time()

        if not configg.multiple_runs:
            print(" | ".join([f"Epoch: {epoch:4d}", f"Train loss: {train_loss.item():.3f}",
                              f"Train acc: {train_acc:.2f}",
                              f"Val loss: {val_loss.item():.3f}",
                              f"Val acc: {val_acc:.2f}"]))

        if configg.use_early_stopping and stopped_early:
            break

    if (not configg.multiple_runs) and configg.use_early_stopping and stopped_early:
        print(f"EARLY STOPPING condition met. Stopped at epoch: {epoch}.")
    else:
        t_end = time.time()

    if not configg.multiple_runs:
        print(f"Total training time: {t_end-t_start:.2f} seconds")

    # print(type(validation_acc))
    # print(validation_acc)

    # print(type(validation_loss))
    # print(validation_loss)

    return validation_acc, validation_loss


In [6]:
def evaluate_on_test(model, features, labels, adj, test_ind, configg):
    if configg.cuda:
        model.cuda()
        adj = adj.cuda()
        features = features.cuda()
        labels = labels.cuda()

    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        model.eval()
        y_pred = model(features, adj)
        test_loss = criterion(y_pred[test_ind], labels[test_ind])
        test_acc = accuracy(y_pred[test_ind], labels[test_ind])

    if not configg.multiple_runs:
        print()
        print(f"Test loss: {test_loss:.3f}  |  Test acc: {test_acc:.2f}")
        return y_pred
    else:
        return test_acc.item(), test_loss.item()

In [7]:
def multiple_runs(model, features, labels, adj, indices, configg, training_loop, evaluate_on_test):
    train_set_ind, val_set_ind, test_set_ind = indices
    acc = []
    loss = []

    t1 = time.time()
    for i in range(configg.num_of_runs):
        print("Run:", i+1)
        model.initialize_weights()
        training_loop(model, features, labels, adj,
                      train_set_ind, val_set_ind, configg)

        acc_curr, loss_curr = evaluate_on_test(model, features, labels,
                                               adj, test_set_ind, configg)
        acc.append(acc_curr)
        loss.append(loss_curr)

    print(f"ACC:  mean: {np.mean(acc):.2f} | std: {np.std(acc):.2f}")
    print(f"LOSS: mean: {np.mean(loss):.2f} | std: {np.std(loss):.2f}")
    print(f"Total training time: {time.time()-t1:.2f} seconds")


In [8]:
# Color map for each class
cora_label_to_color_map = {0: "red", 1: "blue", 2: "green",
                           3: "orange", 4: "yellow", 5: "pink", 6: "gray"}


def visualize_embedding_tSNE(labels, out_features, num_classes):
    """ https://github.com/gordicaleksa/pytorch-GAT """
    node_labels = labels.cpu().numpy()
    out_features = out_features.cpu().numpy()
    t_sne_embeddings = TSNE(n_components=2, perplexity=30, method='barnes_hut').fit_transform(out_features)

    plt.figure()
    for class_id in range(num_classes):
        plt.scatter(t_sne_embeddings[node_labels == class_id, 0],
                    t_sne_embeddings[node_labels == class_id, 1], s=20,
                    color=cora_label_to_color_map[class_id],
                    edgecolors='black', linewidths=0.15)

    plt.axis("off")
    plt.title("t-SNE projection of the learned features")
    plt.show()


def visualize_graph(edges, node_labels, save=False):
    """ Most of the code within this function was taken and "fine-tuned"
        from the Aleksa Gordić's repo:
        https://github.com/gordicaleksa/pytorch-GAT
    """
    num_of_nodes = len(node_labels)
    edge_index_tuples = list(zip(edges[:, 0], edges[:, 1]))

    ig_graph = ig.Graph()
    ig_graph.add_vertices(num_of_nodes)
    ig_graph.add_edges(edge_index_tuples)

    # Prepare the visualization settings dictionary
    visual_style = {"bbox": (1000, 1000), "margin": 50}

    # Normalization of the edge weights
    edge_weights_raw = np.clip(np.log(np.asarray(ig_graph.edge_betweenness()) + 1e-16), a_min=0, a_max=None)
    edge_weights_raw_normalized = edge_weights_raw / np.max(edge_weights_raw)
    edge_weights = [w/3 for w in edge_weights_raw_normalized]
    visual_style["edge_width"] = edge_weights

    # A simple heuristic for vertex size. Multiplying with 0.75 gave decent visualization
    visual_style["vertex_size"] = [0.75*deg for deg in ig_graph.degree()]

    visual_style["vertex_color"] = [cora_label_to_color_map[label] for label in node_labels]

    # Display the cora graph
    visual_style["layout"] = ig_graph.layout_kamada_kawai()
    out = ig.plot(ig_graph, **visual_style)

    if save:
        out.save("cora_visualized.png")


def visualize_validation_performance(val_acc, val_loss):
    val_acc = [v.cpu().item() if isinstance(v, torch.Tensor) else v for v in val_acc]
    val_loss = [v.cpu().item() if isinstance(v, torch.Tensor) else v for v in val_loss]

    f, axs = plt.subplots(1, 2, figsize=(13, 5.5))
    axs[0].plot(val_loss, linewidth=2, color="red")
    axs[0].set_title("Validation loss")
    axs[0].set_ylabel("Cross Entropy Loss")
    axs[0].set_xlabel("Epoch")
    axs[0].grid()

    axs[1].plot(val_acc, linewidth=2, color="red")
    axs[1].set_title("Validation accuracy")
    axs[1].set_ylabel("Acc")
    axs[1].set_xlabel("Epoch")
    axs[1].grid()

    plt.show()


In [9]:
!wandb login 3b0311b8e53fccf34f947a23cec3783cb54aa5c0

sweep_config = {
    'method': 'random',
}

parameters_dict = {
    'lrval': {
        'values': [1, 0.1, 0.01, 0.001]
      },
    'optimizerval': {
        'values': ['Adam', 'Adadelta']
        },
    'epochh': {
        'values': [100, 150, 200]
    }
}

sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="dl-gcn-project-final-run")

Traceback (most recent call last):
  File "/home/vg2507/.local/bin/wandb", line 5, in <module>
    from wandb.cli.cli import cli
  File "/home/vg2507/.local/lib/python3.11/site-packages/wandb/__init__.py", line 22, in <module>
    from wandb.errors import Error
ModuleNotFoundError: No module named 'wandb.errors'
{'method': 'random',
 'parameters': {'epochh': {'values': [100, 150, 200]},
                'lrval': {'values': [1, 0.1, 0.01, 0.001]},
                'optimizerval': {'values': ['Adam', 'Adadelta']}}}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: xnresywg
Sweep URL: https://wandb.ai/team9449/dl-gcn-project-final-run/sweeps/xnresywg


In [10]:
features, labels, adj, edges = load_data(configg)
visualize_graph(edges, labels.cpu().tolist(), save=False)
NUM_CLASSES = int(labels.max().item() + 1)

train_set_ind, val_set_ind, test_set_ind = prepare_dataset(labels, NUM_CLASSES, configg)

def sweepFunction(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = GCN(features.shape[1], configg.hidden_dim, NUM_CLASSES, configg.dropout, configg.use_bias)
        val_acc, val_loss = training_loop(model, features, labels, adj, train_set_ind, val_set_ind, configg, config.epochh, config.optimizerval, config.lrval)
        out_features = evaluate_on_test(model, features, labels, adj, test_set_ind, configg)
        
        # visualize_validation_performance(val_acc, val_loss)
        # visualize_embedding_tSNE(labels, out_features, NUM_CLASSES)

wandb.agent(sweep_id, sweepFunction, count=16)

Loading Cora dataset...
Dataset loaded.


wandb: Agent Starting Run: i6lyk655 with config:
wandb: 	epochh: 200
wandb: 	lrval: 0.001
wandb: 	optimizerval: Adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vishgoki (team9449). Use `wandb login --relogin` to force relogin


Epoch:    0 | Train loss: 1.956 | Train acc: 0.14 | Val loss: 1.924 | Val acc: 0.31
Epoch:    1 | Train loss: 1.944 | Train acc: 0.17 | Val loss: 1.917 | Val acc: 0.31
Epoch:    2 | Train loss: 1.931 | Train acc: 0.19 | Val loss: 1.915 | Val acc: 0.30
Epoch:    3 | Train loss: 1.916 | Train acc: 0.24 | Val loss: 1.907 | Val acc: 0.30
Epoch:    4 | Train loss: 1.908 | Train acc: 0.24 | Val loss: 1.900 | Val acc: 0.31
Epoch:    5 | Train loss: 1.897 | Train acc: 0.29 | Val loss: 1.889 | Val acc: 0.37
Epoch:    6 | Train loss: 1.882 | Train acc: 0.34 | Val loss: 1.884 | Val acc: 0.36
Epoch:    7 | Train loss: 1.862 | Train acc: 0.41 | Val loss: 1.870 | Val acc: 0.45
Epoch:    8 | Train loss: 1.846 | Train acc: 0.43 | Val loss: 1.863 | Val acc: 0.44
Epoch:    9 | Train loss: 1.839 | Train acc: 0.42 | Val loss: 1.854 | Val acc: 0.48
Epoch:   10 | Train loss: 1.834 | Train acc: 0.44 | Val loss: 1.843 | Val acc: 0.45
Epoch:   11 | Train loss: 1.811 | Train acc: 0.49 | Val loss: 1.825 | Val ac

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_acc,▁▂▃▅▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇█████████████████████
training_loss,███▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_acc,▁▂▃▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█▇▇▇██▇▇███▇████▇████
val_loss,███▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁
epoch,163
training_acc,0.97143
training_loss,0.36868
val_acc,0.756
val_loss,0.85516


wandb: Agent Starting Run: h16ro818 with config:
wandb: 	epochh: 100
wandb: 	lrval: 0.001
wandb: 	optimizerval: Adadelta
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

Epoch:    0 | Train loss: 1.939 | Train acc: 0.20 | Val loss: 1.956 | Val acc: 0.13
Epoch:    1 | Train loss: 1.942 | Train acc: 0.16 | Val loss: 1.957 | Val acc: 0.12
Epoch:    2 | Train loss: 1.936 | Train acc: 0.19 | Val loss: 1.953 | Val acc: 0.12
Epoch:    3 | Train loss: 1.937 | Train acc: 0.17 | Val loss: 1.955 | Val acc: 0.11
Epoch:    4 | Train loss: 1.935 | Train acc: 0.18 | Val loss: 1.956 | Val acc: 0.12
Epoch:    5 | Train loss: 1.932 | Train acc: 0.19 | Val loss: 1.954 | Val acc: 0.14
Epoch:    6 | Train loss: 1.935 | Train acc: 0.22 | Val loss: 1.954 | Val acc: 0.12
Epoch:    7 | Train loss: 1.936 | Train acc: 0.12 | Val loss: 1.950 | Val acc: 0.14
Epoch:    8 | Train loss: 1.932 | Train acc: 0.16 | Val loss: 1.950 | Val acc: 0.12
Epoch:    9 | Train loss: 1.930 | Train acc: 0.23 | Val loss: 1.952 | Val acc: 0.13
Epoch:   10 | Train loss: 1.924 | Train acc: 0.22 | Val loss: 1.947 | Val acc: 0.15
Epoch:   11 | Train loss: 1.923 | Train acc: 0.21 | Val loss: 1.952 | Val ac

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_acc,▁▁▁▁▂▂▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▆▇▇▆▇▇▇▇▇███▇▇█
training_loss,██████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁
val_acc,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█▇
val_loss,██████▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▂▂▂▂▂▁▁
epoch,199
training_acc,0.86429
training_loss,1.31705
val_acc,0.582
val_loss,1.58869


wandb: Agent Starting Run: lsns8dml with config:
wandb: 	epochh: 200
wandb: 	lrval: 0.1
wandb: 	optimizerval: Adadelta
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch:    0 | Train loss: 1.935 | Train acc: 0.16 | Val loss: 1.952 | Val acc: 0.13
Epoch:    1 | Train loss: 1.940 | Train acc: 0.14 | Val loss: 1.952 | Val acc: 0.15
Epoch:    2 | Train loss: 1.931 | Train acc: 0.16 | Val loss: 1.948 | Val acc: 0.13
Epoch:    3 | Train loss: 1.932 | Train acc: 0.18 | Val loss: 1.945 | Val acc: 0.14
Epoch:    4 | Train loss: 1.924 | Train acc: 0.20 | Val loss: 1.945 | Val acc: 0.15
Epoch:    5 | Train loss: 1.928 | Train acc: 0.18 | Val loss: 1.948 | Val acc: 0.14
Epoch:    6 | Train loss: 1.919 | Train acc: 0.23 | Val loss: 1.944 | Val acc: 0.15
Epoch:    7 | Train loss: 1.925 | Train acc: 0.21 | Val loss: 1.942 | Val acc: 0.15
Epoch:    8 | Train loss: 1.916 | Train acc: 0.26 | Val loss: 1.943 | Val acc: 0.14
Epoch:    9 | Train loss: 1.921 | Train acc: 0.19 | Val loss: 1.942 | Val acc: 0.13
Epoch:   10 | Train loss: 1.915 | Train acc: 0.23 | Val loss: 1.943 | Val acc: 0.16
Epoch:   11 | Train loss: 1.914 | Train acc: 0.24 | Val loss: 1.939 | Val ac

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
training_acc,▁▁▂▂▃▃▃▃▃▄▄▄▄▅▄▅▅▅▅▅▆▇▆▆▇▆▇▇▇▇█▇▇▇█▇▇███
training_loss,████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
val_acc,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▇▆▆▇▆▇▇█▇▇███
val_loss,██████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▂▂▂▂▂▁▁▁
epoch,199
training_acc,0.85
training_loss,1.24974
val_acc,0.622
val_loss,1.52343
